THIS DOES NOT WORK DUE TO LACK OF FLASH ATTENTION SUPPORT ON WINDOWS.

In [3]:
import os
print(os.environ.get('CUDA_HOME'))

C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1


In [2]:
# import os
# os.environ['CUDA_HOME'] = r'C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1'

# !pip install torch==2.3.0
# !pip install wheel
# !pip install flash_attn==2.5.8
# !pip install numpy==1.24.4
# !pip install Pillow==10.3.0
# !pip install Requests==2.31.0
# !pip install torchvision==0.18.0
# !pip install transformers==4.40.2

  Obtaining dependency information for torch==2.3.0 from https://files.pythonhosted.org/packages/2e/f7/503bab04f4e7a0a43f2ff05c3635cee9dfcf2e09656020d29502b87a94a3/torch-2.3.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/41/24/0b023b6537dfc9bae2c779353998e3e99ac7dfff4222fc6126650e93c3f3/filelock-3.14.0-py3-none-any.whl.metadata
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/d2/05/e6600db80270777c4a64238a98d442f0fd07cc8915be2a1c16da7f2b9e74/sympy-1.12-py3-none-any.whl.metadata
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/38/e9/5f72929373e1a0e8d142a130f3f97e6ff920070f87f91c4e13e40e0fba5a/networkx-3.3-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec from https://files.pyt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached flash_attn-2.5.8.tar.gz (2.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [20 lines of output]
  fatal: not a git repository (or any of the parent directories): .git
  C:\Users\jason\AppData\Local\Temp\pip-install-gwo13q66\flash-attn_e51ff9ee00b64b5d86d3bbc6b6417eef\setup.py:78: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
    warnings.warn(
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\jason\AppData\Local\Temp\pip-install-gwo13q66\flash-attn_e51ff9ee00b64b5d86d3bbc6b6417eef\setup.py", line 134, in <module>
      CUDAExtension(
    File "C:\shared-drive\dev\python-playground\venv\lib\site-packages\torch\utils\cpp_extension.py", 

In [1]:
from PIL import Image 
import requests 
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

model_path = "C:\\shared-drive\\llm_models\\Phi-3-vision-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="cuda", trust_remote_code=True, torch_dtype="auto")

processor_path = "C:\\shared-drive\\llm_models\\Phi-3-vision-128k-instruct"
processor = AutoProcessor.from_pretrained(processor_path, trust_remote_code=True)

messages = [ 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}, 
    {"role": "assistant", "content": "The chart displays the percentage of respondents who agree with various statements about their preparedness for meetings. It shows five categories: 'Having clear and pre-defined goals for meetings', 'Knowing where to find the information I need for a meeting', 'Understanding my exact role and responsibilities when I'm invited', 'Having tools to manage admin tasks like note-taking or summarization', and 'Having more focus time to sufficiently prepare for meetings'. Each category has an associated bar indicating the level of agreement, measured on a scale from 0% to 100%."}, 
    {"role": "user", "content": "Provide insightful questions to spark discussion."} 
] 

url = "https://assets-c4akfrf5b4d3f4b7.z01.azurefd.net/assets/2024/04/BMDataViz_661fb89f3845e.png" 
image = Image.open(requests.get(url, stream=True).raw) 

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0") 

generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

print(response)


ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`